# RTEM Dashboard
Author: SS  
Date: 10/05/2022  
Description: Notebook to explore the structure of the RTEM API and plot useful charts  

## Import libraries

In [1]:
# Standard python libraries
import pandas as pd
import json
import numpy as np

In [2]:
# RTEM libraries
from onboard.client import RtemClient
from onboard.client.dataframes import points_df_from_streaming_timeseries
from onboard.client.models import PointSelector, TimeseriesQuery, PointData
from datetime import datetime, timezone, timedelta

## Load secret
This is done so we dont share API keys.   
To get this working on your notebook, create a folder called 'secrets' in your root directory and create a 'secrets.txt' inside it with your API key.

In [3]:
# Read secret from /secrets/ folder
with open('secrets/secrets.txt') as f:
    secret = f.readlines()[0]

# Setup client
client = RtemClient(api_key=secret)

In [7]:
# Run this once to minimise making too many calls
df_buildings = pd.DataFrame(client.get_all_buildings())

In [8]:
df_buildings.head()

,id,org_id,name,address,sq_ft,image_src,bms_manufacturer,bms_product_name,bms_version,timezone,info,status,equip_count,point_count
0,441,5,88737,None,551143.0,None,None,None,None,America/New_York,"{'org': '', 'floors': '', 'm2fend': '', 'saten...",LIVE,20,1293
1,140,5,83483,None,21869.0,None,None,None,None,America/New_York,"{'org': '', 'floors': '', 'm2fend': '', 'saten...",LIVE,13,354
2,191,5,94414,None,198057.0,None,None,None,None,America/New_York,"{'org': '', 'floors': '', 'm2fend': '23:00', '...",LIVE,7,19
3,231,5,89496,None,NaN,None,None,None,None,America/New_York,"{'org': '', 'floors': '', 'm2fend': '', 'saten...",LIVE,4,51
4,248,5,116742,None,127000.0,None,None,None,None,America/New_York,"{'org': '', 'floors': '', 'm2fend': '', 'saten...",LIVE,7,34


In [14]:
print(f'shape of dataframe {df_buildings.shape}')
df_buildings.describe()

shape of dataframe (229, 14)


,id,org_id,sq_ft,equip_count,point_count
count,229.000000,229.0,2.010000e+02,229.000000,229.000000
mean,279.296943,5.0,3.598326e+05,26.572052,193.842795
std,126.752691,0.0,5.004452e+05,50.564882,510.273853
min,98.000000,5.0,7.960000e+03,1.000000,1.000000
25%,165.000000,5.0,9.000000e+04,4.000000,22.000000
50%,259.000000,5.0,1.980570e+05,8.000000,38.000000
75%,392.000000,5.0,3.640000e+05,25.000000,182.000000
max,503.000000,5.0,3.000000e+06,393.000000,6068.000000


### Something interesting going on in the info column

In [17]:
df_buildings['info'][0]

{'org': '',
 'floors': '',
 'm2fend': '',
 'satend': '',
 'sunend': '',
 'geoCity': 'Bronx',
 'geoState': 'NY',
 'm2fstart': '',
 'satstart': '',
 'sunstart': '',
 'yearBuilt': '',
 'geoCountry': 'US',
 'weatherRef': '',
 'customerType': 'Multifamily'}

In [29]:
# splitting these values into columns in the df
objs = [df_buildings, pd.DataFrame(df_buildings['info'].tolist()).iloc[:, :14]]
df_buildings_flt = pd.concat(objs, axis=1).drop('info', axis=1)

# Plotting

In [36]:
df_buildings_flt.columns

Index(['id', 'org_id', 'name', 'address', 'sq_ft', 'image_src',
       'bms_manufacturer', 'bms_product_name', 'bms_version', 'timezone',
       'status', 'equip_count', 'point_count', 'org', 'floors', 'm2fend',
       'satend', 'sunend', 'geoCity', 'geoState', 'm2fstart', 'satstart',
       'sunstart', 'yearBuilt', 'geoCountry', 'weatherRef', 'customerType'],
      dtype='object')

In [34]:
# Import plottting libraries
import plotly
import seaborn as sns
pd.options.plotting.backend = "plotly" 

In [38]:
df_buildings_flt[['sq_ft']].plot(kind='hist',title = "Histogram of building area")

In [48]:
df_buildings_flt.plot(kind='scatter',
x = 'sq_ft',
y = 'customerType',
title = "Building Typology by area",
c='geoCity')

In [56]:
df_buildings_flt[["customerType"]].plot(kind='bar', title = "Counts of buildings by curtomer type")

## Now some function to help select buildings

In [58]:
def sb_buildings_by_function(df,function_string):
    """Function to get all buildings from the all buildings dataframe and filter them by function
    params:
    df: A pandas dataframe
    function_string : A list of string functions
    return: Filtered pandas dataframe"""
    filtered_df = df[df["customerType"]==function_string]
    return filtered_df
    

In [61]:
df_multifamily = sb_buildings_by_function(df_buildings_flt,"Multifamily")
df_multifamily.head()

,id,org_id,name,address,sq_ft,image_src,bms_manufacturer,bms_product_name,bms_version,timezone,...,sunend,geoCity,geoState,m2fstart,satstart,sunstart,yearBuilt,geoCountry,weatherRef,customerType
0,441,5,88737,None,551143.0,None,None,None,None,America/New_York,...,,Bronx,NY,,,,,US,,Multifamily
4,248,5,116742,None,127000.0,None,None,None,None,America/New_York,...,,New York,NY,,,,,US,,Multifamily
6,434,5,129666,None,56090.0,None,None,None,None,America/New_York,...,,Brooklyn,NY,,,,,US,,Multifamily
11,317,5,128286,None,35000.0,None,None,None,None,America/New_York,...,,New York,NY,,,,,US,,Multifamily
13,375,5,97789,None,232808.0,None,None,None,None,America/New_York,...,,BRONX,NY,,,,,US,,Multifamily


## Going into the building sensors now